<a href="https://colab.research.google.com/github/aihubgrw/Talha-Malik-s-Repo/blob/main/Result_Card_System_with_Admin_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Result Card System BY **Talha Younus**

In [2]:
import pandas as pd
from IPython.display import display, Markdown
from termcolor import colored

# Subjects offered
subjects = ["English", "Urdu", "Math", "Bio"]

# Name validation (alphabets and space only)
def is_valid_name(name):
    return all(c.isalpha() or c.isspace() for c in name)

# Numeric input validation
def get_valid_number(prompt):
    while True:
        try:
            return int(input(prompt))
        except:
            print("❌ Only numbers are allowed.")

# Terminal-style colored output
def bold_color(text, color='cyan'):
    colors = ['black','red','green','yellow','blue','magenta','cyan','white']
    color_index = 30 + colors.index(color)
    return f"\033[1;{color_index}m{text}\033[0m"

# Input: class and student info
class_name = input("Enter the class name: ")
num_students = get_valid_number("Enter total number of students: ")
total_marks_per_subject = get_valid_number("Enter total marks per subject: ")
total_marks = total_marks_per_subject * len(subjects)

# Initialize storage
passed_students = []
failed_students = []
subject_totals = {subj: 0 for subj in subjects}

# Collect student data
for i in range(num_students):
    print(f"\n--- Enter details for Student {i+1} ---")

    while True:
        name = input("Enter name of student: ").strip()
        if is_valid_name(name):
            break
        else:
            print("❌ Only alphabets and spaces are allowed.")

    marks = {}
    failed_subjects = []

    for subject in subjects:
        while True:
            try:
                mark = int(input(f"Enter obtained marks in {subject}: "))
                if 0 <= mark <= total_marks_per_subject:
                    marks[subject] = mark
                    if mark < (0.33 * total_marks_per_subject):
                        failed_subjects.append(subject)
                    break
                else:
                    print(f"❌ Marks must be between 0 and {total_marks_per_subject}")
            except:
                print("❌ Invalid number. Try again.")

    student = {"name": name, "marks": marks}

    if failed_subjects:
        student["failed"] = True
        student["failed_subjects"] = failed_subjects
        failed_students.append(student)
    else:
        student["failed"] = False
        total = sum(marks.values())
        percentage = (total / total_marks) * 100

        # Grade assignment
        if percentage >= 80:
            grade = "A+"
        elif percentage >= 70:
            grade = "A"
        elif percentage >= 60:
            grade = "B"
        elif percentage >= 50:
            grade = "C"
        elif percentage >= 33:
            grade = "D"
        else:
            grade = "F"

        # Equal marks check
        unique_scores = set(marks.values())
        if len(unique_scores) == 1:
            highest_subjects = lowest_subjects = ["Equal Marks in all subjects"]
        else:
            max_score = max(marks.values())
            min_score = min(marks.values())
            highest_subjects = [subj for subj, score in marks.items() if score == max_score]
            lowest_subjects = [subj for subj, score in marks.items() if score == min_score]

        student.update({
            "total": total,
            "percentage": percentage,
            "grade": grade,
            "highest_subjects": highest_subjects,
            "lowest_subjects": lowest_subjects
        })

        # Subject totals for admin report
        for subj, score in marks.items():
            subject_totals[subj] += score

        passed_students.append(student)

# Assign positions to passed students
passed_students.sort(key=lambda x: x["total"], reverse=True)
for idx, s in enumerate(passed_students):
    s["position"] = idx + 1

# Combine all students
all_students_sorted = passed_students + failed_students

# 🎓 Display all result cards
for student in all_students_sorted:
    display(Markdown(f"### 📝 Result Card: **{student['name']}** — Class: **{class_name}**"))

    # Marks table (vertical)
    marks_df = pd.DataFrame.from_dict(student["marks"], orient='index', columns=["Marks"])
    display(marks_df.style.set_properties(**{'font-weight': 'bold', 'color': 'blue'}))

    if student["failed"]:
        for subj in student["failed_subjects"]:
            needed = int(0.33 * total_marks_per_subject)
            print(bold_color(f"❌ Fail in {subj}. You needed to secure at least {needed} marks in {subj}.", "red"))
        print(bold_color("⚠️ Result: Better luck next time!", "red"))
    else:
        print(bold_color(f"✅ Total: {student['total']} / {total_marks}", "green"))
        print(bold_color(f"📊 Percentage: {student['percentage']:.2f}%", "green"))
        print(bold_color(f"🎖️ Grade: {student['grade']}", "green"))
        print(bold_color(f"🏅 Position: {student['position']}", "green"))

        if student["highest_subjects"][0] == "Equal Marks in all subjects":
            print(bold_color("📚 All subjects have equal marks", "yellow"))
        else:
            print(bold_color(f"📈 Highest Marks In: {', '.join(student['highest_subjects'])}", "yellow"))
            print(bold_color(f"📉 Lowest Marks In: {', '.join(student['lowest_subjects'])}", "yellow"))

    print("\n" + "-"*60)

# 🏆 Topper
if passed_students:
    topper = passed_students[0]
    print(bold_color(f"\n🏆 Topper of Class {class_name}: {topper['name']} with {topper['percentage']:.2f}%", "magenta"))
else:
    print(bold_color("\nNo student passed. No topper this time.", "red"))

# 📋 Admin Report: Subject-wise average percentages
print(bold_color(f"\n📋 Admin Report: Subject-wise Average Percentages", "blue"))

if passed_students:
    averages = {}
    for subj in subjects:
        avg_mark = subject_totals[subj] / len(passed_students)
        avg_percent = (avg_mark / total_marks_per_subject) * 100
        averages[subj] = round(avg_percent, 2)

    admin_df = pd.DataFrame.from_dict(averages, orient='index', columns=['Average %'])
    display(admin_df.style.set_properties(**{'font-weight': 'bold', 'color': 'purple'}))
else:
    print("No passed students to calculate averages.")


Enter the class name: 1
Enter total number of students: 1
Enter total marks per subject: 3

--- Enter details for Student 1 ---
Enter name of student: A
Enter obtained marks in English: 1
Enter obtained marks in Urdu: 1
Enter obtained marks in Math: 2
Enter obtained marks in Bio: 3


### 📝 Result Card: **A** — Class: **1**

,Marks
English,1
Urdu,1
Math,2
Bio,3


✅ Total: 7 / 12
📊 Percentage: 58.33%
🎖️ Grade: C
🏅 Position: 1
📈 Highest Marks In: Bio
📉 Lowest Marks In: English, Urdu

------------------------------------------------------------

🏆 Topper of Class 1: A with 58.33%

📋 Admin Report: Subject-wise Average Percentages


,Average %
English,33.330000
Urdu,33.330000
Math,66.670000
Bio,100.000000


In [3]:
# 📋 Admin Report: Subject-wise Stats
print(bold_color(f"\n📋 Admin Report: Subject-wise Statistics", "blue"))

subject_stats = {}

# Initialize counters
for subj in subjects:
    subject_stats[subj] = {
        "Average %": 0,
        "Passed": 0,
        "Failed": 0,
        "Total Appeared": num_students
    }

# Count pass/fail and sum marks
for student in all_students_sorted:
    for subj in subjects:
        mark = student["marks"][subj]
        if not student["failed"]:  # Count average only for passed students
            subject_stats[subj]["Average %"] += mark
        if subj in student.get("failed_subjects", []):
            subject_stats[subj]["Failed"] += 1
        else:
            subject_stats[subj]["Passed"] += 1

# Finalize averages
for subj in subjects:
    passed_count = len(passed_students)
    if passed_count > 0:
        avg = subject_stats[subj]["Average %"] / passed_count
        subject_stats[subj]["Average %"] = round((avg / total_marks_per_subject) * 100, 2)
    else:
        subject_stats[subj]["Average %"] = "N/A"

# Display
admin_df = pd.DataFrame.from_dict(subject_stats, orient='index')
display(admin_df.style.set_properties(**{'font-weight': 'bold', 'color': 'purple'}))



📋 Admin Report: Subject-wise Statistics


,Average %,Passed,Failed,Total Appeared
English,33.330000,1,0,1
Urdu,33.330000,1,0,1
Math,66.670000,1,0,1
Bio,100.000000,1,0,1
